In [162]:
import pandas as pd, sqlite3

# Ex04. A/B testing

## 1. Подключение к базе данных

In [163]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
cursor = conn.cursor()

## 2. Проверка гипотезы

*Со страницей "Лента новостей" студенты быстрее приступают к выполнению лабораторных работ.*

### 2.1. Разница между последним коммитом и дедлайном ДО и ПОСЛЕ посещения ленты новостей

**Тестовая группа**

In [164]:
cursor.execute('''
               WITH uids AS (
                SELECT DISTINCT(t1.uid)
                FROM test t1
                JOIN test t2 ON t1.uid = t2.uid
                WHERE t1.first_commit_ts > t1.first_view_ts AND t2.first_commit_ts < t2.first_view_ts 
                AND t1.labname != 'project1' AND t2.labname != 'project1'
               )
        
               SELECT AVG(avg_after)
               FROM
                (SELECT t.uid, AVG((strftime('%s', t.first_commit_ts) - d.deadlines) / 3600) avg_after
                FROM test t
                JOIN deadlines d ON d.labs = t.labname
                JOIN uids u ON u.uid = t.uid
                WHERE labname != 'project1' AND first_commit_ts > first_view_ts
                GROUP BY t.uid)
               
               UNION
               
               SELECT AVG(avg_before)
               FROM
                (SELECT t.uid, AVG((strftime('%s', t.first_commit_ts) - d.deadlines) / 3600) avg_before
                FROM test t
                JOIN deadlines d ON d.labs = t.labname
                JOIN uids u ON u.uid = t.uid
                WHERE labname != 'project1' AND first_commit_ts < first_view_ts
                GROUP BY t.uid)
               ''')
test_results = pd.DataFrame({'time': ['after', 'before'], 
                             'avg_diff': [row[0] for row in cursor.fetchall()]})
test_results

,time,avg_diff
0,after,-99.523810
1,before,-66.047619


**Контрольная группа**

In [165]:
cursor.execute('''
               WITH uids AS (
                SELECT DISTINCT(c1.uid)
                FROM control c1
                JOIN control c2 ON c1.uid = c2.uid
                WHERE c1.first_commit_ts > c1.first_view_ts AND c2.first_commit_ts < c2.first_view_ts 
                AND c1.labname != 'project1' AND c2.labname != 'project1'
               )
               
               SELECT AVG(avg_after)
               FROM
                (SELECT c.uid, AVG((strftime('%s', c.first_commit_ts) - d.deadlines) / 3600) avg_after
                FROM control c
                JOIN deadlines d ON d.labs = c.labname
                JOIN uids u ON u.uid = c.uid
                WHERE labname != 'project1' AND first_commit_ts > first_view_ts
                GROUP BY c.uid)
               
               UNION 

               SELECT AVG(avg_before)
               FROM
                (SELECT c.uid, AVG((strftime('%s', c.first_commit_ts) - d.deadlines) / 3600) avg_before
                FROM control c
                JOIN deadlines d ON d.labs = c.labname
                JOIN uids u ON u.uid = c.uid
                WHERE labname != 'project1' AND first_commit_ts < first_view_ts
                GROUP BY c.uid)
               ''')
control_results = pd.DataFrame({'time': ['after', 'before'], 
                             'avg_diff': [row[0] for row in cursor.fetchall()]})
control_results

,time,avg_diff
0,after,-99.322222
1,before,-98.033333


*Дельта до первого посещения новостной ленты существенно отличается от дельты после него в тестовой группе, но в контрольной группе мы не видим того же эффекта, следовательно гипотеза **верна** и студенты быстрее приступают к выполнению лабораторных.*

## 3. Закрытие соединения

In [166]:
conn.close()